In [ ]:
import ase
import ase.io
import ase.visualize
import numpy as np

import matplotlib.pyplot as plt

import tb_mean_field_hubbard as tbmfh

In [ ]:
# GEOMETRY
xyz_file = "clars_goblet.xyz"
geom = ase.io.read(xyz_file)

# Spin guess is set in atom tags (1 - up, 2 - down)
# one option to modify it is with the following (edit->modify & file->save):
#ase.visualize.view(geom)

In [ ]:
# CALCULATION PARAMETERS

t1 = -2.7 # first nearest neighbor
t2 = -0.1 # second nearest neighbor
t3 = -0.4 # third nearest neighbor

t_list = [t1, t2, t3]

charge = 0
multiplicity = 1

In [ ]:
mfh_model = tbmfh.MeanFieldHubbardModel(geom, t_list, charge, multiplicity)

mfh_model.print_parameters()

mfh_model.visualize_spin_guess()
#mfh_model.visualize_tb_model()

In [ ]:
mfh_model.run_mfh(u = 3.0, print_iter=False, plot=False)

In [ ]:
mfh_model.report(num_orb=1)

In [ ]:
# Visualize one specific orbital

spin = 0
index = 20
print("Orbital index: %d, spin %d" % (index, spin))
print("Energy: %.6f eV" % mfh_model.evals[spin][index])
mfh_model.plot_orbital(mo_index=index, spin=spin)

# corresponding eigenvector (each element corresponds in order to atoms defined in geom/mfh_model.ase_geom)
evec = mfh_model.evecs[spin][index]

In [ ]:
# Series in U

u_list = np.arange(0.0, 5.0, 0.1)
abs_mag_list = []
energy_list = []
gap_list = []

for u in u_list:
    mfh_model.run_mfh(u = u, print_iter=False, plot=False)
    abs_mag_list.append(mfh_model.mag_list[-1])
    energy_list.append(mfh_model.energy_list[-1])
    gap_list.append(mfh_model.gap_eff)

In [ ]:
plt.plot(u_list, abs_mag_list, 'o')
plt.ylabel("Absolute magnetization")
plt.xlabel("U [eV]")
plt.show()